In [52]:
# installing necessary packages

#!pip install pandas azure-storage-blob dotenv

In [53]:
# importing necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os 

In [54]:
# Data Extraction
try:
    data = pd.read_csv('zipco_transaction.csv')
    print('Data Extracted successfully!')
except Exception as e:
    print(f"an error occured : {e}")

Data Extracted successfully!


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [56]:
# remove duplicates
data.drop_duplicates(inplace=True)

In [57]:
# Handle missing values( filling missing numeric values with mean and median)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()},inplace=True)
    
    

In [58]:
# Handle missing values (where there is string/object with unknown)
string_column = data.select_dtypes(include=['object']).columns
for col in string_column:
    data.fillna({col: 'unknown'},inplace=True)

    data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        900 non-null    object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [59]:
data['Date']= pd.to_datetime(data['Date'])

In [60]:
# Creating dataframe tables 
products = data[['ProductName', 'UnitPrice']].drop_duplicates().reset_index(drop=True)
# add product ID to the table
products.index.name = 'Product_ID'
products = products.reset_index()
products.head(15)

,Product_ID,ProductName,UnitPrice
0,0,Vanilla Cake,12.532304
1,1,Red Velvet Cake,7.083070
2,2,Chocolate Cake,6.736064
3,3,Carrot Cake,7.314823
4,4,Pizza Pepperoni,7.577727
5,5,Red Velvet Cake,14.568156
6,6,Red Velvet Cake,16.114719
7,7,Carrot Cake,12.949873
8,8,Carrot Cake,17.435266
9,9,Pizza Margherita,19.868948


In [61]:
# Customers table
customers = data[['CustomerName', 'CustomerAddress','Customer_PhoneNumber','CustomerEmail']].drop_duplicates().reset_index(drop=True)
# add product ID to the table
customers.index.name = 'Customer_ID'
# make product id the first column on the table
customers = customers.reset_index()
customers.head(15)

,Customer_ID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net
5,5,Curtis Cummings,"25468 Eric Causeway Suite 343\r\nEast Steven, ...",(647)294-0826,stephanie69@example.net
6,6,Haley Wilcox,"PSC 3319, Box 1711\r\nAPO AE 54478",001-345-422-5023,sharon86@example.net
7,7,Tara Martinez,"49488 Rachel Port\r\nAnthonybury, FL 31446",267.413.3199x19807,unknown
8,8,Sarah Carter,74613 Hensley Crossroad Suite 746\r\nPort Kell...,(742)946-6553x159,anthony69@example.org
9,9,Abigail Simon,"57371 Wilson Alley\r\nPatriciaburgh, PA 36781",213.290.9906x79391,lawrence51@example.com


In [62]:
# Staff table
Staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
# add product ID to the table
Staff.index.name = 'Staff_ID'
Staff = Staff.reset_index()
Staff.head(15)

,Staff_ID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org
5,5,Jessica Coleman,sandra03@example.org
6,6,Kelly Johnston,edwardsaaron@example.org
7,7,Rebecca George,michaelperez@example.org
8,8,Sara Aguilar,nicholasmullins@example.org
9,9,Michelle Pace,freemanjose@example.com


In [63]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [64]:
# Transactions Table
Transaction = data.merge(products, on=['ProductName', 'UnitPrice'],how = 'left')\
                  .merge(customers, on=['CustomerName', 'CustomerAddress','Customer_PhoneNumber','CustomerEmail'],how = 'left')\
                  .merge(Staff, on=['Staff_Name', 'Staff_Email',],how = 'left')
Transaction.index.name = 'Transaction_ID'
Transaction = Transaction.reset_index()\
                         [['Date','Transaction_ID','Product_ID','Customer_ID','Staff_ID','Quantity','OrderType',\
                           'StoreLocation','PaymentType', 'PromotionApplied', 'Weather', 'Temperature',\
                           'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'DayOfWeek','TotalSales']]

In [66]:
# Save data as csv
data.to_csv('clean_data.csv', index= False)
Transaction.to_csv('Transaction.csv', index= False)
products.to_csv('products.csv', index= False)
customers.to_csv('customers.csv', index= False)
Staff.to_csv('Staff.csv', index= False)

In [67]:
# Data loading to azure
# load the environment variables from .env
load_dotenv()
connect_str = os.getenv('CONNECT_STR')
container_name = os.getenv('CONTAINER_NAME')

In [ ]:
# Create Blobservice client object

blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# load data into azure blob storage
files = [
    (data, 'rawdata/cleaned_zipco_Transaction_data.csv'), # creating a folder inside the created contanier
    (products, 'cleandata/products.csv'),
    (customers, 'cleandata/customers.csv'),
    (Staff, 'cleandata/Staff.csv'),
    (Transaction, 'cleandata/Transaction.csv'),
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into azure storage')